## Overview

The Content Creation Assistant notebook harnesses cutting-edge artificial intelligence to streamline and enhance digital content creation. It integrates advanced tools and frameworks to offer a comprehensive solution for both written and visual content generation.

At its core is the LlamaIndex framework, which efficiently manages complex workflows and integrates various AI models and APIs. The Tavily Search API provides real-time, relevant information to ensure that the content is engaging, informative, and timely.

For text generation, the notebook utilizes the OpenAI GPT model, known for producing coherent and contextually accurate text, ideal for blog posts and social media updates. Complementing this, the OpenAI DALL-E 3 model generates unique images based on the themes of the text, enhancing visual appeal and reader engagement.

Overall, the Content Creation Assistant empowers users—whether bloggers, marketers, or social media managers—to produce rich, multi-dimensional content effortlessly, saving time and boosting creativity.

In [1]:
!pip install llama-index-core llama-index-utils-workflow asyncio llama-index-llms-openai tavily-python openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 51.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.0/795.0 kB 46.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


### Import Necessary Libraries

In [2]:
import getpass
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Context,
    Event
)
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.openai import OpenAI
from tavily import TavilyClient
from pydantic import BaseModel, Field
from typing import Annotated
from openai import OpenAI as dalle3 # use specifically for image generation
from PIL import Image
from io import BytesIO
import requests
import uuid
import os
import asyncio

### Set Environment Variables

In [3]:
tavily_api_key = getpass.getpass()
openai_api_key = getpass.getpass()

 ········
 ········


## Setting up Utils

In [4]:
class TavilySearchInput(BaseModel):
    query: Annotated[str, Field(description="The search query string")]
    max_results: Annotated[
        int, Field(description="Maximum number of results to return", ge=1, le=10)
    ] = 5
    search_depth: Annotated[
        str,
        Field(
            description="Search depth: 'basic' or 'advanced'",
            choices=["basic", "advanced"],
        ),
    ] = "basic"

def tavily_search(query: Annotated[TavilySearchInput, "Input for Tavily search"]):
    tavily_client = TavilyClient(api_key=tavily_api_key)
    # Perform the search
    response = tavily_client.search(
        query=query.query,
        max_results=query.max_results,
        search_depth=query.search_depth,
    )

    # Format the results
    formatted_results = []
    for result in response.get("results", []):
        formatted_results.append(
            f"Title: {result['title']}\\nURL: {result['url']}\\nContent: {result['content']}\\n"
        )

    return "\\n".join(formatted_results)

def save_file(content, uuid, type="Blog"):
        # Define the filename for the markdown file
        directory = uuid
        file_name = f"{type}_{uuid}.md"
        file_path = os.path.join(directory, file_name)
    
        try:
            # Create the directory if it doesn't exist
            os.makedirs(directory, exist_ok=True)
    
            # Open the file in write mode ('w') and save the blog content
            with open(file_path, 'w') as file:
                file.write(f"# {content}")
            print(f"{type} post saved to {file_path}.")
        except Exception as e:
            print("An error occurred while saving the blog post:", e)

async def generate_image_with_retries(client, prompt, max_retries=3, delay=2):
    attempt = 0
    while attempt < max_retries:
        try:
            # Attempt to generate the image using the dalle3 API
            response = client.images.generate(
                model="dall-e-3",
                prompt=prompt,
                size="1024x1024",
                quality="standard",
                n=1,
            )
            return response
        except Exception as e:
            attempt += 1
            print(f"Attempt {attempt} failed with error: {e}")
            if attempt < max_retries:
                await asyncio.sleep(delay)
                print(f"Retrying after {delay} seconds...")
            else:
                print("Exceeded maximum retries.")
                raise

## Setup Prompt Instructions

In [5]:
from prompts.prompts import *

blog_template = BLOG_TEMPLATE
blog_and_research_template = BLOG_AND_RESEARCH_TEMPLATE
image_prompt_instructions = IMAGE_GENERATION_TEMPLATE
linked_in_template = LINKED_IN_TEMPLATE

## Setting up Workflow

In [6]:
class ResearchEvent(Event):
    query: str
    uuid: str

class BlogEvent(Event):
    query: str
    research: str
    uuid: str

class BlogWithoutResearch(Event):
    query: str
    uuid: str

class SocialMediaEvent(Event):
    blog: str
    uuid: str

class SocialMediaCompleteEvent(Event):
    result: str

class IllustratorEvent(Event):
    blog: str
    
class IllustratorCompleteEvent(Event):
    result: str

In [7]:
class ContentCreationWorkflow(Workflow):

    @step
    async def start(self, ctx: Context, ev: StartEvent) -> ResearchEvent | BlogWithoutResearch :
        print("Starting content creation", ev.query)
        id = str(uuid.uuid4())
        if (ev.research) is False:
            return BlogWithoutResearch(query=ev.query, uuid=id)
        return ResearchEvent(query=ev.query, uuid=id)

    @step
    async def step_research(self, ctx: Context, ev: ResearchEvent) -> BlogEvent:
        print("Researching users query")
        search_input = TavilySearchInput(
            query=ev.query,
            max_results=3,
            search_depth="basic")
        research = tavily_search(search_input)
        return BlogEvent(query=ev.query, research=research, uuid=ev.uuid)

    @step
    async def step_blog_without_research(self, ctx: Context, ev: BlogWithoutResearch) -> SocialMediaEvent | IllustratorEvent:
        print("Writing blog post without research")
        print("uuid", ev.uuid)
        llm = OpenAI(model="gpt-4o-mini", api_key=openai_api_key)
        prompt = blog_template.format(query_str=ev.query)
        result = await llm.acomplete(prompt, formatted=True)
        save_file(result.text, ev.uuid)
        print(result)
        ctx.send_event(SocialMediaEvent(blog=result.text, uuid=ev.uuid))
        ctx.send_event(IllustratorEvent(blog=result.text, uuid=ev.uuid))
                        
    @step
    async def step_blog(self, ctx: Context, ev: BlogEvent) -> SocialMediaEvent | IllustratorEvent:
        print("Writing blog post")

        llm = OpenAI(model="gpt-4o-mini", api_key=openai_api_key)
        prompt = blog_and_research_template.format(query_str=ev.query, research=ev.research)
        result = await llm.acomplete(prompt, formatted=True)

        save_file(result.text, ev.uuid)
        ctx.send_event(SocialMediaEvent(blog=result.text, uuid=ev.uuid))
        ctx.send_event(IllustratorEvent(blog=result.text, uuid=ev.uuid))

    @step
    async def step_social_media(self, ctx: Context, ev: SocialMediaEvent) -> SocialMediaCompleteEvent:
        print("Writing social media post")
        llm = OpenAI(model="gpt-4o-mini", api_key=openai_api_key)
        prompt = linked_in_template.format(blog_content=ev.blog)
        results = await llm.acomplete(prompt, formatted=True)
        save_file(results.text, ev.uuid, type="LinkedIn")
        return SocialMediaCompleteEvent(result="LinkedIn post written")

    @step
    async def step_illustrator(self, ctx: Context, ev:IllustratorEvent) -> IllustratorCompleteEvent:
        print("Generating image")
        llm = OpenAI(model="gpt-4o-mini", api_key=openai_api_key)
        image_prompt_instruction_generator = image_prompt_instructions.format(blog_post=ev.blog)
        image_prompt = await llm.acomplete(image_prompt_instruction_generator, formatted=True)
        
        client = dalle3(api_key=openai_api_key)
        response = await generate_image_with_retries(client, image_prompt.text)
        image_url = response.data[0].url
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content))
        
        directory = f'./{ev.uuid}'
        os.makedirs(directory, exist_ok=True)
        image.save(f'{directory}/generated_image.png')
        image.save(f'{ev.uuid}/generated_image.png')
        
        return IllustratorCompleteEvent(result="Images drawn")

    @step
    async def step_collection(self, ctx: Context, ev: SocialMediaCompleteEvent | IllustratorCompleteEvent) -> StopEvent:
        if (
            ctx.collect_events(
                ev,
                [SocialMediaCompleteEvent, IllustratorCompleteEvent]
            ) is None
        ) : return None
        return StopEvent(result="Done")

In [8]:
w = ContentCreationWorkflow(timeout=120, verbose=False)
result = await w.run(query="Albert Einstein", research=True)
print(result)

Starting content creation Albert Einstein
Researching users query
Writing blog post
Blog post saved to 093e7f9e-d409-4e70-adc8-75397962ea05/Blog_093e7f9e-d409-4e70-adc8-75397962ea05.md.
Writing social media post
LinkedIn post saved to 093e7f9e-d409-4e70-adc8-75397962ea05/LinkedIn_093e7f9e-d409-4e70-adc8-75397962ea05.md.
Done
